In [ ]:

import numpy as np
from scipy.linalg import qr, lu, ldl, cholesky, solve
from scipy.sparse.linalg import splu
import matplotlib.pyplot as plt
import sympy as sp
from 



# 1.3

# LU dense

In [25]:
import numpy as np
import time
import psutil
from scipy.linalg import lu_factor, lu_solve, ldl
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu, spsolve
import qdldl
#from qdldl import factor

def measure_cpu_usage(interval=1.0):
    return psutil.cpu_percent(interval=interval)

def EqualityQPSolverLUdense(H, g, A, b):
    start_time = time.time()
    start_cpu = measure_cpu_usage(interval=1.0)
    
    n = H.shape[0]
    m = A.shape[0]
    KKT = np.block([
        [H, -A.T],
        [A, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-g, b])
    lu, piv = lu_factor(KKT)
    sol = lu_solve((lu, piv), rhs)
    x = sol[:n]
    lam = sol[n:]
    
    end_time = time.time()
    end_cpu = measure_cpu_usage(interval=1.0)
    
    elapsed_time = end_time - start_time
    cpu_usage = end_cpu - start_cpu
    
    return x, lam, elapsed_time, cpu_usage

def EqualityQPSolverLUsparse(H, g, A, b):
    start_time = time.time()
    start_cpu = measure_cpu_usage(interval=1.0)
    
    n = H.shape[0]
    m = A.shape[0]
    Hs = csc_matrix(H)
    As = csc_matrix(A)
    KKT = csc_matrix(np.block([
        [Hs.toarray(), -As.T.toarray()],
        [As.toarray(), np.zeros((m, m))]
    ]))
    rhs = np.concatenate([-g, b])
    lu = splu(KKT)
    sol = lu.solve(rhs)
    x = sol[:n]
    lam = sol[n:]
    
    end_time = time.time()
    end_cpu = measure_cpu_usage(interval=1.0)
    
    elapsed_time = end_time - start_time
    cpu_usage = end_cpu - start_cpu
    
    return x, lam, elapsed_time, cpu_usage

def EqualityQPSolverLDLdense(H, g, A, b):
    start_time = time.time()
    start_cpu = measure_cpu_usage(interval=1.0)
    
    n = H.shape[0]
    m = A.shape[0]
    KKT = np.block([
        [H, -A.T],
        [A, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-g, b])
    L, D, perm = ldl(KKT)
    y = np.linalg.solve(L, rhs[perm])
    z = np.linalg.solve(D, y)
    sol = np.linalg.solve(L.T, z)
    x = sol[:n]
    lam = sol[n:]
    
    end_time = time.time()
    end_cpu = measure_cpu_usage(interval=1.0)
    
    elapsed_time = end_time - start_time
    cpu_usage = end_cpu - start_cpu
    
    return x, lam, elapsed_time, cpu_usage

def EqualityQPSolverLDLsparse(H, g, A, b):
 #doesnt work
 return ('hallo')

def EqualityQPSolverRangeSpace(H, g, A, b):
    start_time = time.time()
    start_cpu = measure_cpu_usage(interval=1.0)
    
    # Range space method
    At = A.T
    S = A @ np.linalg.solve(H, At)
    rhs = b + A @ np.linalg.solve(H, g)
    lam = np.linalg.solve(S, rhs)
    x = -np.linalg.solve(H, g - At @ lam)
    
    end_time = time.time()
    end_cpu = measure_cpu_usage(interval=1.0)
    
    elapsed_time = end_time - start_time
    cpu_usage = end_cpu - start_cpu
    
    return x, lam, elapsed_time, cpu_usage

def EqualityQPSolverNullSpace(H, g, A, b):
    start_time = time.time()
    start_cpu = measure_cpu_usage(interval=1.0)
    
    # Null space method
    from scipy.linalg import null_space
    Z = null_space(A)
    x0 = np.linalg.lstsq(A, b, rcond=None)[0]
    Hz = Z.T @ H @ Z
    gz = Z.T @ (H @ x0 + g)
    p = -np.linalg.solve(Hz, gz)
    x = x0 + Z @ p
    lam = np.linalg.lstsq(A.T, H @ x + g, rcond=None)[0]
    
    end_time = time.time()
    end_cpu = measure_cpu_usage(interval=1.0)
    
    elapsed_time = end_time - start_time
    cpu_usage = end_cpu - start_cpu
    
    return x, lam, elapsed_time, cpu_usage

def EqualityQPSolver(H, g, A, b, solver):
    solvers = {
        "LUdense": EqualityQPSolverLUdense,
        "LUsparse": EqualityQPSolverLUsparse,
        "LDLdense": EqualityQPSolverLDLdense,
        "LDLsparse": EqualityQPSolverLDLsparse,
        "RangeSpace": EqualityQPSolverRangeSpace,
        "NullSpace": EqualityQPSolverNullSpace
    }
    if solver not in solvers:
        raise ValueError(f"Unknown solver: {solver}")
    return solvers[solver](H, g, A, b)

def generate_test_problem(n, beta, alpha):
    # Generate random matrix M with 15% nonzero elements
    M = np.random.randn(n, n)
    mask_M = np.random.rand(n, n) > 0.85
    M[mask_M] = 0
    
    # Generate H = M M' + alpha I
    H = M @ M.T + alpha * np.eye(n)
    
    # Generate random vector g
    g = np.random.randn(n)
    
    # Generate matrix A with 15% nonzero elements
    m = round(beta * n)
    A = np.random.randn(m, n)
    mask_A = np.random.rand(m, n) > 0.85
    A[mask_A] = 0
    
    # Generate random vector b
    b = np.random.randn(m)
    
    return H, g, A, b

def compute_lagrangian(H, g, A, b, x, lam):
    L = 0.5 * x.T @ H @ x + g.T @ x - lam.T @ (A @ x - b)
    return L

def EqualityQPSolver(H, g, A, b, solver):
    solvers = {
        "LUdense": EqualityQPSolverLUdense,
        "LUsparse": EqualityQPSolverLUsparse,
        "LDLdense": EqualityQPSolverLDLdense,
        "LDLsparse": EqualityQPSolverLDLsparse,
        "RangeSpace": EqualityQPSolverRangeSpace,
        "NullSpace": EqualityQPSolverNullSpace
    }
    if solver not in solvers:
        raise ValueError(f"Unknown solver: {solver}")
    return solvers[solver](H, g, A, b)


In [26]:
def generate_test_problem(n, beta, alpha):
    # Generate random matrix M with 15% nonzero elements
    M = np.random.randn(n, n)
    mask_M = np.random.rand(n, n) > 0.85
    M[mask_M] = 0
    
    # Generate H = M M' + alpha I
    H = M @ M.T + alpha * np.eye(n)
    
    # Generate random vector g
    g = np.random.randn(n)
    
    # Generate matrix A with 15% nonzero elements
    m = round(beta * n)
    A = np.random.randn(m, n)
    mask_A = np.random.rand(m, n) > 0.85
    A[mask_A] = 0
    
    # Generate random vector b
    b = np.random.randn(m)
    
    return H, g, A, b

def compute_lagrangian(H, g, A, b, x, lam):
    L = 0.5 * x.T @ H @ x + g.T @ x - lam.T @ (A @ x - b)
    return L




In [27]:
# Parameters
n = 100  # Size of x
beta = 0.5  # Ratio for size of A
alpha = 0.1  # Regularization parameter

# Generate test problem
H, g, A, b = generate_test_problem(n, beta, alpha)

In [28]:
# x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'LDLsparse')
# L = compute_lagrangian(H, g, A, b, x, lam)
# L, time1, cpu1 

In [29]:
x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'LDLdense')
L = compute_lagrangian(H, g, A, b, x, lam)
L, time1, cpu1 

(np.float64(25.464013736485082), 1.0047433376312256, -4.0)

In [30]:
x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'LUdense')
L = compute_lagrangian(H, g, A, b, x, lam)
L, time1, cpu1



(np.float64(26.168487717375417), 1.0038247108459473, 3.4000000000000004)

In [31]:
x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'LUsparse')
L = compute_lagrangian(H, g, A, b, x, lam)
L, time1, cpu1


(np.float64(26.168487717375413), 1.0072195529937744, -1.8999999999999995)

In [32]:
x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'RangeSpace')
L = compute_lagrangian(H, g, A, b, x, lam)
L, time1, cpu1

(np.float64(26.16848771737542), 1.0008697509765625, -8.7)

In [33]:
x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'NullSpace')
L = compute_lagrangian(H, g, A, b, x, lam)
L, time1, cpu1

(np.float64(26.16848771737542), 1.0064151287078857, 0.7000000000000002)